In [ ]:
"""Programme forward pour modéliser les temps d'arrivée de la réflexion sur le bulbe lors d'une injection de type Porchet
suivie par radar de sol"""

%matplotlib inline

import numpy as np
import os
#import matplotlib.pyplot as plt
#import sys, h5py, binascii

import pygimli as pg
from pygimli.meshtools import appendTriangleBoundary, merge2Meshes, mesh
from pygimli.mplviewer import drawMesh
from pygimli.viewer import showMesh

from definition_maillage import *
from ecriture_fich_gprMax import *
from ecriture_in import *
from interp_charge_trou import *
from interp_maillage_gprMax import *

#Def des paramètres de géométrie du modéle
xmin,xmax = 0, 40 # en cm
emin,emax = 0, 80 #  elevation en cm
dtrou = 30 # prof du trou en cm
etrou = emax - dtrou # elevation du fond du trou
r=2 # rayon du trou en cm
dx = .1 #On définit le pas de la maille
zaff= 20 #profondeur en cm jusqu'où on souhaite un maillage affiné. 
eaff=etrou-zaff

h_0=-95 #charge initiale en cm, soit l'état initial du sol (teneur en eau exprimée en charge)
h_1=10 #hauteur d'eau au fond du trou en cm

#Définition du maillage triangulaire
[mesh, pg_pos, mesh_pos, mesh_cells]=definition_maillage(xmin, xmax, emin, emax, dtrou, etrou, r, dx, zaff, eaff)

#Début de la boucle de calcul
for i in range(0,1):
    tr = 0.06
    ts = 0.3
    alpha = 0.016
    n = 8.52
    Ks = 0.07+i*0.01
    param = [tr, ts, tr, ts, alpha, n, Ks, Ks, ts] #Paramètres d'entrée tr, ts, alpha, n, Ks
    temps=[0.17, 0.33, 0.50, 0.67, 0.83, 1.00, 1.17, 1.33, 1.50, 1.67, 1.83, 2.00, 2.17, 2.33, 2.50, 2.67, 2.83, 3.00, 3.17, 3.33, 3.50, 3.67, 3.83, 4.00, 4.17, 4.33, 4.50, 4.67, 4.83, 5.00, 5.17, 5.33, 5.50, 5.67, 5.83, 6.00]#Temps de calcul en minutes
    #temps=[1.00, 2.00, 3.00, 4.00, 5.00, 6.00]#Temps de calcul en minutes
    
    #Interpolation de la charge sur les bords du trou
    [p, charge_imp, n_free_drainage, n_total]=interp_charge_trou(mesh_pos, etrou, r, h_0, h_1)

    #Création des fichiers .in nécessaires pour SMWS_2D
    ecriture_in(mesh, param, temps, p, n_total)

    #Lancement SWMS_2D
    !cp Grid.in SWMS_2D.IN/
    !cp Selector.in SWMS_2D.IN/
    !~/Codes/HD2/H2D

    #Interpolation du maillage sur une grille rectangulaire 2D pour gprMax
    #On crée un maillage rectangulaire avec les dimensions du modèle
    xmin, xmax = 0.0, 40.
    zmin, zmax = 0.0, 80.

    dx = 1.0 #en cm

    #fich_thetas = "TEST_AFF/th.out" #Fichier contenant les thetas
    fich_thetas = "SWMS_2D.OUT/th.out" #Fichier contenant les thetas

    [xv, yv, T, mx, my, mesh2, grid, grid_mat, eps_mat] = interp_maillage_gprMax(xmin, xmax, zmin, zmax, dx, param, mesh, mesh_pos[:,:2], fich_thetas, temps)

    import math
    #Lancement gprMax
    nom='Forward'
   
    sigma=0.0000
    #eps_w=80.1 #Epsilon eau
    eps_w=80.1
    eps_pvc=3 #Epsilon pvc
    eps_s=2.5 #Epsilon sable
    p=param[1] #Porosité = theta s
    #Maxieps=( math.sqrt(eps_w)*0.9*p + (1-p)*math.sqrt(eps_s) +0.1*p )**2
    Maxieps=( math.sqrt(eps_w)*p + (1-p)*math.sqrt(eps_s) )**2
    Minv=0.3/math.sqrt(Maxieps)
    #Minv=0.3/eps_w
    Minlambda=Minv*(10**9)/(2800*(10**6))
    dl=Minlambda/10

    time=0.000000030
    dx=abs((xv.T*0.01)[0,1]-(xv.T*0.01)[0,0])
    dy=abs((yv.T*0.01)[1,0]-(yv.T*0.01)[0,0])
    #dl=dx/2
    #dl=dx
    fac_dt = 0.2

    #Définition de ce qu'on a besoin pour gprMax
    import os
    import math
    from scipy.interpolate import griddata

    #for i in range(0,len(T)) :
    #    grid_z0 = griddata((mx, my), grid[:,i], (xv, yv), method='linear', fill_value=0.0).T
    
    materiaux = {}

    def materiau(x):
        if x in materiaux:
            return materiaux[x]
        valeur = "sand{}".format(len(materiaux))
        materiaux[x] = valeur
        return valeur
    for i in range(0,len(T)+1):
        for j in range(0,81):
            for k in range(0,41):
                #grid_mat[i][j,k]=round(grid_mat[i][j,k],3)
                #grid_mat[i][j,k]=grid_mat[i][j,k]*2
                #f[i][j,k]=CRIM(grid_mat[i][j,k], eps_w,eps_s,p)
                materiau(grid_mat[i][j,k])
        #nmedia=len((yv.T*0.01)[:,0])*len(grid_mat[i][0,:])
        nmedia=len(materiaux)+2
        ecriture_fich_gprMax(param, xv.T*0.01, yv.T*0.01, T, grid_mat[i], i, param[7], nom, h_1*0.01, etrou*0.01, r*0.01, sigma, eps_w, eps_pvc, eps_s, p, nmedia, time, dx, dy, dl, fac_dt, materiaux)
        fichier=nom+str(int(param[7]*1000))+'_'+str(i+1)+'.in'
        command="../gprMax "+fichier
        os.popen(command).readlines()
    
    #os.popen("rm -rf *.in")
        #Lancement du logiciel gprMax
    #!../gprMax Forward70_28.in
    #!bash bash.sh
    #!rm -rf *.in

 Example                                                                 

 Program SWMS_2D
 Time independent boundary conditions
 Axisymmetric flow, V = L*L*L
 Units: L = cm   , T = min  , M = -    
 -----------------------------------------------------
 |                                                   |
 |                     SWMS_2D                       |
 |                                                   |
 |     Code for simulating water flow and solute     |
 |       transport in two-dimensional variably       |
 |             saturated porous media                |
 |                                                   |
 |                  version 1.22                     |
 |          Last modified: January, 1994             |
 |                                                   |
 -----------------------------------------------------

 Example                                                                 
 Axisymmetric flow, V = L*L*L
 reading nodal information
 reading

        1.9363  4   451  -0.280E+03 -0.688E+03  0.000E+00    10.     0.
        1.9575  4   455  -0.279E+03 -0.694E+03  0.000E+00    10.     0.
        1.9787  4   459  -0.279E+03 -0.700E+03  0.000E+00    10.     0.
        2.0000  4   463  -0.278E+03 -0.706E+03  0.000E+00    10.     0.
        2.0213  4   467  -0.278E+03 -0.712E+03  0.000E+00    10.     0.
        2.0425  4   471  -0.277E+03 -0.718E+03  0.000E+00    10.     0.
        2.0638  3   474  -0.277E+03 -0.724E+03  0.000E+00    10.     0.
        2.0903  4   478  -0.276E+03 -0.731E+03  0.000E+00    10.     0.
        2.1169  4   482  -0.276E+03 -0.739E+03  0.000E+00    10.     0.
        2.1434  4   486  -0.275E+03 -0.746E+03  0.000E+00    10.     0.
        2.1700  4   490  -0.274E+03 -0.753E+03  0.000E+00    10.     0.
        2.1929  4   494  -0.274E+03 -0.759E+03  0.000E+00    10.     0.
        2.2157  4   498  -0.274E+03 -0.766E+03  0.000E+00    10.     0.
        2.2386  4   502  -0.273E+03 -0.772E+03  0.000E+00    10.

        5.7767  4   895  -0.243E+03 -0.167E+04  0.000E+00    10.     0.
        5.8300  4   899  -0.243E+03 -0.169E+04  0.000E+00    10.     0.
        5.8867  4   903  -0.243E+03 -0.170E+04  0.000E+00    10.     0.
        5.9433  4   907  -0.243E+03 -0.172E+04  0.000E+00    10.     0.
        6.0000  4   911  -0.242E+03 -0.173E+04  0.000E+00    10.     0.


In [23]:
test = "ls -al "+"*.in"
test = "rm -rf "+"Forward70_3.in"
import os
os.popen(test).readlines()

[]

In [35]:
nom+str(int(param[7]*1000))+'_'+str(i+1)+'.in'

'Forward70_1.in'